<a href="https://colab.research.google.com/github/mzsmarcos/projeto_pipelineETL/blob/main/Etapa_1_c%C3%B3digo_extra%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Extração e leitura inicial dos arquivos
O código a seguir, faz a extração automática dos arquivos de empresas do site da Receita Federal, salva os arquivos em uma pasta do Google Drive.

É feita também a inspeção inicial dos arquivos**

In [ ]:
#Criando a pasta do Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Baixar todos os arquivos do site da Receita

import os
from bs4 import BeautifulSoup
import time
import requests
from random import randint

# URL do diretório onde os arquivos estão hospedados
base_url = "https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/2025-01/"

# Caminho onde os arquivos serão salvos no Google Drive
pasta_destino = "/content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos"

# Criar a pasta se não existir
os.makedirs(pasta_destino, exist_ok=True)

def listar_arquivos(url):
    """Obtém a lista de arquivos disponíveis no diretório da Receita Federal."""
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    arquivos = []
    for link in soup.find_all("a"):
        href = link.get("href")
        if href.endswith(".zip"):  # Filtrar apenas arquivos ZIP
            arquivos.append(href)

    return arquivos

def baixar_arquivos(arquivos, url, destino, max_retries=5):
    """Baixa os arquivos ZIP se ainda não estiverem salvos, com tentativas em caso de erro."""
    for arquivo in arquivos:
        caminho_arquivo = os.path.join(destino, arquivo)

        if os.path.exists(caminho_arquivo):
            print(f"🔹 {arquivo} já existe. Pulando...")
            continue  # Pula arquivos já baixados

        tentativa = 0
        while tentativa < max_retries:
            try:
                print(f"⬇️ Tentando baixar {arquivo}... (Tentativa {tentativa+1}/{max_retries})")
                response = requests.get(url + arquivo, stream=True)
                response.raise_for_status()  # Verifica se houve erro na resposta

                with open(caminho_arquivo, "wb") as f:
                    for chunk in response.iter_content(chunk_size=1024):
                        if chunk:
                            f.write(chunk)
                print(f"✅ {arquivo} baixado com sucesso!")
                break  # Sai do loop de tentativas após sucesso
            except requests.exceptions.RequestException as e:
                print(f"Erro ao baixar {arquivo}: {e}")
                tentativa += 1
                if tentativa < max_retries:
                    # Atraso exponencial entre as tentativas
                    wait_time = randint(5, 10)  # Atraso entre 5 e 10 segundos
                    print(f"Esperando {wait_time} segundos antes da próxima tentativa...")
                    time.sleep(wait_time)
                else:
                    print(f"❌ Não foi possível baixar {arquivo} após {max_retries} tentativas.")

            time.sleep(2)  # Pausa de 2 segundos entre os downloads para evitar sobrecarga

# Listar os arquivos disponíveis na URL
arquivos_para_baixar = listar_arquivos(base_url)

# Baixar os arquivos na pasta do Google Drive
baixar_arquivos(arquivos_para_baixar, base_url, pasta_destino)

print("\n🎉 Download concluído!")


⬇️ Tentando baixar Cnaes.zip... (Tentativa 1/5)
✅ Cnaes.zip baixado com sucesso!
⬇️ Tentando baixar Empresas0.zip... (Tentativa 1/5)
✅ Empresas0.zip baixado com sucesso!
⬇️ Tentando baixar Empresas1.zip... (Tentativa 1/5)
✅ Empresas1.zip baixado com sucesso!
⬇️ Tentando baixar Empresas2.zip... (Tentativa 1/5)
✅ Empresas2.zip baixado com sucesso!
⬇️ Tentando baixar Empresas3.zip... (Tentativa 1/5)
✅ Empresas3.zip baixado com sucesso!
⬇️ Tentando baixar Empresas4.zip... (Tentativa 1/5)
✅ Empresas4.zip baixado com sucesso!
⬇️ Tentando baixar Empresas5.zip... (Tentativa 1/5)
✅ Empresas5.zip baixado com sucesso!
⬇️ Tentando baixar Empresas6.zip... (Tentativa 1/5)
✅ Empresas6.zip baixado com sucesso!
⬇️ Tentando baixar Empresas7.zip... (Tentativa 1/5)
✅ Empresas7.zip baixado com sucesso!
⬇️ Tentando baixar Empresas8.zip... (Tentativa 1/5)
✅ Empresas8.zip baixado com sucesso!
⬇️ Tentando baixar Empresas9.zip... (Tentativa 1/5)
✅ Empresas9.zip baixado com sucesso!
⬇️ Tentando baixar Estabeleci

# **Os códigos a seguir fazem a leitura inicial dos arquivos CSV zipados apenas para ter a ideia de como os dados estão estruturados.**

In [ ]:
#Bibliotecas iniciais

import pandas as pd
from google.colab import drive
import zipfile
import os

In [ ]:
# Leitura do arquivo Cnaes

# Montar o Google Drive
drive.mount('/content/drive')

# Caminho do arquivo ZIP
zip_path = "/content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/Cnaes.zip"


# Inspecionar o conteúdo do ZIP
with zipfile.ZipFile(zip_path, 'r') as z:
    z.printdir()  # Isso mostra os arquivos dentro do ZIP
    csv_filename = z.namelist()[0]  # Pegamos o primeiro arquivo do ZIP

# Ler o CSV dentro do ZIP
df_cnaes = pd.read_csv(zip_path, compression='zip', sep=';', encoding='latin1', dtype=str, nrows=1000)

# Exibir as primeiras linhas
df_cnaes.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File Name                                             Modified             Size
F.K03200$Z.D50111.CNAECSV                      2025-01-11 14:46:02        88215


,0111301,Cultivo de arroz
0,0111302,Cultivo de milho
1,0111303,Cultivo de trigo
2,0111399,Cultivo de outros cereais não especificados an...
3,0112101,Cultivo de algodão herbáceo
4,0112102,Cultivo de juta


In [ ]:
df_cnaes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   0111301           1000 non-null   object
 1   Cultivo de arroz  1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


In [ ]:
# Leitura do arquivo EmpresasO

# Caminho do arquivo ZIP
zip_path = "/content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/Empresas0.zip"


# Inspecionar o conteúdo do ZIP
with zipfile.ZipFile(zip_path, 'r') as z:
    z.printdir()  # Isso mostra os arquivos dentro do ZIP
    csv_filename = z.namelist()[0]  # Pegamos o primeiro arquivo do ZIP

# Ler o CSV dentro do ZIP
df_empresas0 = pd.read_csv(zip_path, compression='zip', sep=';', encoding='latin1', dtype=str, nrows=1000)

# Exibir as primeiras linhas
df_empresas0.head()



File Name                                             Modified             Size
K3241.K03200Y0.D50111.EMPRECSV                 2025-01-11 16:52:20   1593224816


,41273592,41.273.592 HELIO DE JESUS PEREIRA,2135,50,"30000,00",01,Unnamed: 6
0,41273593,JULIO CESAR NUNES 39611300867,2135,50,"3000,00",01,NaN
1,41273594,OZINETE DELFINO CALDAS 41608224287,2135,50,"5000,00",01,NaN
2,41273595,GILVAN PEREIRA XAVIER 01363090380,2135,50,"3000,00",01,NaN
3,41273596,RODRIGO JOSE FERREIRA LOPES 05010247941,2135,50,"10000,00",01,NaN
4,41273597,PACHARRUS QUEIROZ DA COSTA E SILVA 03618384335,2135,50,"5000,00",01,NaN


In [ ]:
df_empresas0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20506191 entries, 0 to 20506190
Data columns (total 7 columns):
 #   Column                             Dtype 
---  ------                             ----- 
 0   41273592                           object
 1   41.273.592 HELIO DE JESUS PEREIRA  object
 2   2135                               object
 3   50                                 object
 4   30000,00                           object
 5   01                                 object
 6   Unnamed: 6                         object
dtypes: object(7)
memory usage: 1.1+ GB


In [ ]:
# Leitura do arquivo Empresas1

# Caminho do arquivo ZIP
zip_path = "/content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/Empresas1.zip"


# Inspecionar o conteúdo do ZIP
with zipfile.ZipFile(zip_path, 'r') as z:
    z.printdir()  # Isso mostra os arquivos dentro do ZIP
    csv_filename = z.namelist()[0]  # Pegamos o primeiro arquivo do ZIP

# Ler o CSV dentro do ZIP
df_empresas1 = pd.read_csv(zip_path, compression='zip', sep=';', encoding='latin1', dtype=str, nrows=1000)

# Exibir as primeiras linhas
df_empresas1.head()

File Name                                             Modified             Size
K3241.K03200Y1.D50111.EMPRECSV                 2025-01-11 17:37:04    325716169


,00000000,BANCO DO BRASIL SA,2038,10,"120000000000,00",05,Unnamed: 6
0,00000001,ASSOCIACAO DE AMIGOS DE BAIRRO DO CONJ PAULISTANO,3999,16,"0,00",05,NaN
1,00000002,WM&R EMPREITEIRA DE CONSTRUCAO CIVIL LIMITADA,2240,49,"0,00",05,NaN
2,00000003,CASA CARIDADE LUZETE ROBERTA DE MORAIS CONJ PA...,3999,16,"0,00",05,NaN
3,00000004,ASSOCIACAO DO PEQUENO ADOLECENTE DO CONJ PAULI...,3999,16,"0,00",05,NaN
4,00000005,SERRALHERIA PORTA DE ACO REAL S/C LTDA,2240,28,"0,00",05,NaN


In [ ]:
df_empresas1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   00000000            1000 non-null   object
 1   BANCO DO BRASIL SA  1000 non-null   object
 2   2038                1000 non-null   object
 3   10                  1000 non-null   object
 4   120000000000,00     1000 non-null   object
 5   05                  1000 non-null   object
 6   Unnamed: 6          2 non-null      object
dtypes: object(7)
memory usage: 54.8+ KB


In [ ]:
# Leitura do arquivo Estabelecimentos0

# Caminho do arquivo ZIP
zip_path = "/content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/Estabelecimentos0.zip"


# Inspecionar o conteúdo do ZIP
with zipfile.ZipFile(zip_path, 'r') as z:
    z.printdir()  # Isso mostra os arquivos dentro do ZIP
    csv_filename = z.namelist()[0]  # Pegamos o primeiro arquivo do ZIP

# Ler o CSV dentro do ZIP
df_estabelecimentos0 = pd.read_csv(zip_path, compression='zip', sep=';', encoding='latin1', dtype=str, nrows=1000)

# Exibir as primeiras linhas
df_estabelecimentos0.head()


File Name                                             Modified             Size
K3241.K03200Y0.D50111.ESTABELE                 2025-01-11 16:02:14   4936792905


,37220018,0001,09,1,ANTONIO HERCULANO DA MOTA,08,20230530,01,Unnamed: 8,Unnamed: 9,...,7075,11,41014141,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,HERCULANOANTONIO23@GMAIL.COM,Unnamed: 28,Unnamed: 29
0,36068186,0001,68,1,NaN,02,20200120,00,NaN,NaN,...,5035,35,99939878,NaN,NaN,NaN,NaN,ALEVIDROSGL@OUTLOOK.COM,NaN,NaN
1,37318263,0001,53,1,NaN,02,20200604,00,NaN,NaN,...,6377,11,80594733,NaN,NaN,NaN,NaN,VALERIA.POLIANA93@GMAIL.COM,NaN,NaN
2,37364869,0001,25,1,NaN,08,20220831,01,NaN,NaN,...,7075,11,43350869,NaN,NaN,NaN,NaN,PESSOAL1@3ECONTABILIDADE.COM.BR,NaN,NaN
3,37406207,0001,70,1,NaN,02,20200615,00,NaN,NaN,...,7075,11,84027546,NaN,NaN,NaN,NaN,PAULAPIOLA07@GMAIL.COM,NaN,NaN
4,37443430,0001,98,1,NaN,04,20221201,63,NaN,NaN,...,6377,11,93229815,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_estabelecimentos0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   37220018                      1000 non-null   object
 1   0001                          1000 non-null   object
 2   09                            1000 non-null   object
 3   1                             1000 non-null   object
 4   ANTONIO HERCULANO DA MOTA     380 non-null    object
 5   08                            1000 non-null   object
 6   20230530                      1000 non-null   object
 7   01                            1000 non-null   object
 8   Unnamed: 8                    1 non-null      object
 9   Unnamed: 9                    1 non-null      object
 10  20200525                      1000 non-null   object
 11  8712300                       1000 non-null   object
 12  Unnamed: 12                   558 non-null    object
 13  RUA                

In [ ]:
# Leitura do arquivo Estabelecimentos1

# Caminho do arquivo ZIP
zip_path = "/content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/Estabelecimentos1.zip"


# Inspecionar o conteúdo do ZIP
with zipfile.ZipFile(zip_path, 'r') as z:
    z.printdir()  # Isso mostra os arquivos dentro do ZIP
    csv_filename = z.namelist()[0]  # Pegamos o primeiro arquivo do ZIP

# Ler o CSV dentro do ZIP
df_estabelecimentos1 = pd.read_csv(zip_path, compression='zip', sep=';', encoding='latin1', dtype=str, nrows=1000)

# Exibir as primeiras linhas
df_estabelecimentos1.head()

File Name                                             Modified             Size
K3241.K03200Y1.D50111.ESTABELE                 2025-01-11 16:47:36   1075226446


,07396865,0001,68,1,Unnamed: 4,08,20170210,01,Unnamed: 8,Unnamed: 9,...,8297,47,33851125,47.1,33851125.1,47.2,33851125.2,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,64904295,0018,51,2,NaN,08,20161110,01,NaN,NaN,...,2785,11,36491000,31,33880436,82,33118379,CLAUDIO.GIGLIO@CAMIL.COM.BR,NaN,NaN
1,76016369,0003,16,2,NaN,04,20231113,63,NaN,NaN,...,7071,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,52302726,0001,82,1,NaN,04,20210406,63,NaN,NaN,...,6131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,07396923,0001,53,1,NaN,08,20140115,01,NaN,NaN,...,7107,011,69658088,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,03650261,0001,45,1,OTICA PERFEICAO,04,20190322,63,NaN,NaN,...,8113,048,2423953,NaN,NaN,048,2423953,NaN,NaN,NaN


In [ ]:
# Leitura do arquivo Motivos

# Caminho do arquivo ZIP
zip_path = "/content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/Motivos.zip"


# Inspecionar o conteúdo do ZIP
with zipfile.ZipFile(zip_path, 'r') as z:
    z.printdir()  # Isso mostra os arquivos dentro do ZIP
    csv_filename = z.namelist()[0]  # Pegamos o primeiro arquivo do ZIP

# Ler o CSV dentro do ZIP
df_motivos = pd.read_csv(zip_path, compression='zip', sep=';', encoding='latin1', dtype=str)

# Exibir as primeiras linhas
df_motivos.head()


File Name                                             Modified             Size
F.K03200$Z.D50111.MOTICSV                      2025-01-11 14:36:26         2846


,00,SEM MOTIVO
0,01,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA
1,02,INCORPORACAO
2,03,FUSAO
3,04,CISAO TOTAL
4,05,ENCERRAMENTO DA FALENCIA


In [ ]:
df_motivos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   00          60 non-null     object
 1   SEM MOTIVO  60 non-null     object
dtypes: object(2)
memory usage: 1.1+ KB


In [ ]:
# Leitura do arquivo Municipios

# Caminho do arquivo ZIP
zip_path = "/content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/Municipios.zip"


# Inspecionar o conteúdo do ZIP
with zipfile.ZipFile(zip_path, 'r') as z:
    z.printdir()  # Isso mostra os arquivos dentro do ZIP
    csv_filename = z.namelist()[0]  # Pegamos o primeiro arquivo do ZIP

# Ler o CSV dentro do ZIP
df_municipios = pd.read_csv(zip_path, compression='zip', sep=';', encoding='latin1', dtype=str)

# Exibir as primeiras linhas
df_municipios.head()

File Name                                             Modified             Size
F.K03200$Z.D50111.MUNICCSV                     2025-01-11 14:51:00       120403


,0001,GUAJARA-MIRIM
0,0002,ALTO ALEGRE DOS PARECIS
1,0003,PORTO VELHO
2,0004,BURITIS
3,0005,JI-PARANA
4,0006,CHUPINGUAIA


In [ ]:
df_municipios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   0001           5570 non-null   object
 1   GUAJARA-MIRIM  5570 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [ ]:
# Leitura do arquivo Naturezas

# Caminho do arquivo ZIP
zip_path = "/content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/Naturezas.zip"


# Inspecionar o conteúdo do ZIP
with zipfile.ZipFile(zip_path, 'r') as z:
    z.printdir()  # Isso mostra os arquivos dentro do ZIP
    csv_filename = z.namelist()[0]  # Pegamos o primeiro arquivo do ZIP

# Ler o CSV dentro do ZIP
df_naturezas = pd.read_csv(zip_path, compression='zip', sep=';', encoding='latin1', dtype=str)

# Exibir as primeiras linhas
df_naturezas.head()

File Name                                             Modified             Size
F.K03200$Z.D50111.NATJUCSV                     2025-01-11 14:56:00         4141


,0000,Natureza Jurídica não informada
0,3271,Órgão de Direção Local de Partido Político
1,3280,Comitê Financeiro de Partido Político
2,3298,Frente Plebiscitária ou Referendária
3,3301,Organização Social (OS)
4,3999,Associação Privada


In [ ]:
df_naturezas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 2 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   0000                             89 non-null     object
 1   Natureza Jurídica não informada  89 non-null     object
dtypes: object(2)
memory usage: 1.5+ KB


In [ ]:
# Leitura do arquivo Paises

# Caminho do arquivo ZIP
zip_path = "/content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/Paises.zip"


# Inspecionar o conteúdo do ZIP
with zipfile.ZipFile(zip_path, 'r') as z:
    z.printdir()  # Isso mostra os arquivos dentro do ZIP
    csv_filename = z.namelist()[0]  # Pegamos o primeiro arquivo do ZIP

# Ler o CSV dentro do ZIP
df_paises = pd.read_csv(zip_path, compression='zip', sep=';', encoding='latin1', dtype=str)

# Exibir as primeiras linhas
df_paises.head()

File Name                                             Modified             Size
F.K03200$Z.D50111.PAISCSV                      2025-01-11 15:06:00         5444


,000,COLIS POSTAUX
0,013,AFEGANISTAO
1,017,ALBANIA
2,020,"ALBORAN-PEREJIL,ILHAS"
3,023,ALEMANHA
4,025,"ALEMANHA, REP. DEMOCRATICA DA"


In [ ]:
df_paises.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   000            254 non-null    object
 1   COLIS POSTAUX  254 non-null    object
dtypes: object(2)
memory usage: 4.1+ KB


In [ ]:
# Leitura do arquivo Qualificações

# Caminho do arquivo ZIP
zip_path = "/content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/Qualificacoes.zip"


# Inspecionar o conteúdo do ZIP
with zipfile.ZipFile(zip_path, 'r') as z:
    z.printdir()  # Isso mostra os arquivos dentro do ZIP
    csv_filename = z.namelist()[0]  # Pegamos o primeiro arquivo do ZIP

# Ler o CSV dentro do ZIP
df_qualificacoes = pd.read_csv(zip_path, compression='zip', sep=';', encoding='latin1', dtype=str)

# Exibir as primeiras linhas
df_qualificacoes.head()

File Name                                             Modified             Size
F.K03200$Z.D50111.QUALSCSV                     2025-01-11 15:01:00         2355


,00,Não informada
0,05,Administrador
1,08,Conselheiro de Administração
2,09,Curador
3,10,Diretor
4,11,Interventor


In [ ]:
df_qualificacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   00             67 non-null     object
 1   Não informada  67 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


In [ ]:
# Leitura do arquivo Simples

# Caminho do arquivo ZIP
zip_path = "/content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/Simples.zip"


# Inspecionar o conteúdo do ZIP
with zipfile.ZipFile(zip_path, 'r') as z:
    z.printdir()  # Isso mostra os arquivos dentro do ZIP
    csv_filename = z.namelist()[0]  # Pegamos o primeiro arquivo do ZIP

# Ler o CSV dentro do ZIP
df_simples = pd.read_csv(zip_path, compression='zip', sep=';', encoding='latin1', dtype=str, nrows=1000)

# Exibir as primeiras linhas
df_simples.head()

File Name                                             Modified             Size
F.K03200$W.SIMPLES.CSV.D50111                  2025-01-11 14:41:36   2628420732


,00000000,N,20070701,20070701.1,N.1,20090701,20090701.1
0,00000006,N,20180101,20191231,N,00000000,00000000
1,00000008,N,20140101,20211231,N,00000000,00000000
2,00000011,S,20070701,00000000,N,00000000,00000000
3,00000013,N,20090101,20231231,N,00000000,00000000
4,00000015,N,20070701,20081231,N,00000000,00000000


In [ ]:
df_simples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   00000000    1000 non-null   object
 1   N           1000 non-null   object
 2   20070701    1000 non-null   object
 3   20070701.1  1000 non-null   object
 4   N.1         1000 non-null   object
 5   20090701    1000 non-null   object
 6   20090701.1  1000 non-null   object
dtypes: object(7)
memory usage: 54.8+ KB


In [ ]:
# Leitura do arquivo Sócios

# Caminho do arquivo ZIP
zip_path = "/content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/Socios0.zip"


# Inspecionar o conteúdo do ZIP
with zipfile.ZipFile(zip_path, 'r') as z:
    z.printdir()  # Isso mostra os arquivos dentro do ZIP
    csv_filename = z.namelist()[0]  # Pegamos o primeiro arquivo do ZIP

# Ler o CSV dentro do ZIP
df_socios0 = pd.read_csv(zip_path, compression='zip', sep=';', encoding='latin1', dtype=str, nrows=1000)

# Exibir as primeiras linhas
df_socios0.head()

File Name                                             Modified             Size
K3241.K03200Y0.D50111.SOCIOCSV                 2025-01-11 15:11:10    715921325


,66011180,2,CLAUDIO RONDINI,***887038**,49,20011126,Unnamed: 6,***000000**,Unnamed: 8,00,8
0,66011180,2,MARIA APARECIDA QUEIROZ RONDINI,***933348**,22,20050618,NaN,***000000**,NaN,00,7
1,85061877,2,RICARDO BAULE ROSSI,***521929**,49,20060202,NaN,***000000**,NaN,00,4
2,85061877,2,EDUARDO GUILHERME BAULE ROSSI,***550209**,22,20060202,NaN,***000000**,NaN,00,5
3,07428785,2,WENDER ALVES MOREIRA,***544511**,49,20141021,NaN,***000000**,NaN,00,5
4,49250368,2,ELIETE QUEIROZ BARRETO,***614218**,22,20031126,NaN,***000000**,NaN,00,5


In [ ]:
df_socios0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   66011180         1000 non-null   object
 1   2                1000 non-null   object
 2   CLAUDIO RONDINI  999 non-null    object
 3   ***887038**      999 non-null    object
 4   49               1000 non-null   object
 5   20011126         1000 non-null   object
 6   Unnamed: 6       1 non-null      object
 7   ***000000**      1000 non-null   object
 8   Unnamed: 8       23 non-null     object
 9   00               1000 non-null   object
 10  8                1000 non-null   object
dtypes: object(11)
memory usage: 86.1+ KB


In [ ]:
# Leitura do arquivo Sócios1

# Caminho do arquivo ZIP
zip_path = "/content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/Socios1.zip"


# Inspecionar o conteúdo do ZIP
with zipfile.ZipFile(zip_path, 'r') as z:
    z.printdir()  # Isso mostra os arquivos dentro do ZIP
    csv_filename = z.namelist()[0]  # Pegamos o primeiro arquivo do ZIP

# Ler o CSV dentro do ZIP
df_socios1 = pd.read_csv(zip_path, compression='zip', sep=';', encoding='latin1', dtype=str, nrows=1000)

# Exibir as primeiras linhas
df_socios1.head()

File Name                                             Modified             Size
K3241.K03200Y1.D50111.SOCIOCSV                 2025-01-11 15:56:02    206337729


,07396865,2,GERSON HOFFMANN,***240659**,49,20050518,Unnamed: 6,***000000**,Unnamed: 8,00,5
0,07396865,2,MITZI HARTMANN,***126339**,22,20080509,NaN,***000000**,NaN,00,4
1,52302726,2,BENEDITO BEZERRA DE CARVALHO,***061354**,49,20050921,NaN,***000000**,NaN,00,7
2,07396923,2,RENATO PEDUTO,***069508**,49,20050516,NaN,***000000**,NaN,00,6
3,07396923,2,ELAINE GARCIA,***971118**,49,20050516,NaN,***000000**,NaN,00,6
4,03650261,2,DILTO DE SOUZA,***236669**,49,19991217,NaN,***000000**,NaN,00,7


In [ ]:
df_socios1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   07396865         1000 non-null   object
 1   2                1000 non-null   object
 2   GERSON HOFFMANN  1000 non-null   object
 3   ***240659**      1000 non-null   object
 4   49               1000 non-null   object
 5   20050518         1000 non-null   object
 6   Unnamed: 6       3 non-null      object
 7   ***000000**      1000 non-null   object
 8   Unnamed: 8       19 non-null     object
 9   00               1000 non-null   object
 10  5                1000 non-null   object
dtypes: object(11)
memory usage: 86.1+ KB
